In [1]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
from keras.utils import *

In [129]:
import numpy as np

In [130]:
from sklearn.utils import shuffle

In [131]:
import cv2

In [132]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
all_data = np.concatenate((x_train, x_test))

In [133]:
def get_random_datasets(nums, datas, labels, nclasses=10):
    shape_data = list(datas.shape)
    shape_data[0] = nums
    shape_data = tuple(shape_data)
    shape_label = list(labels.shape)
    shape_label[0] = nums
    shape_label = tuple(shape_label)
    data = np.zeros(shape_data)
    label = np.zeros(shape_label)
    cls_num = dict(zip(range(0, nclasses), [0]*nclasses))
    datas, labels = shuffle(datas, labels)
    
    num_total = 0
    for i in range(datas.shape[0]):
        if num_total >= nums:
            break
        num_class = cls_num[labels[i][0]] 
        #print(labels[i][0], num_class)
        if num_class < (nums/nclasses):
            data[num_total] = datas[i]
            label[num_total] = labels[i]
            num_total += 1
            cls_num[labels[i][0]]  += 1
    return data, label

In [134]:
def get_5000_training_data():
    return get_random_datasets(5000,x_train, y_train)

In [135]:
def get_1000_test_data():
    return get_random_datasets(1000,x_test, y_test)

In [136]:
def preprocess_data(datasets):
    #normalize to mean 0 std 1
    gen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True) 
    gen.fit(all_data)
    datasets_norm = (datasets - gen.mean)/gen.std
    #rescale to 224*224
    datasets_resized = np.zeros((datasets.shape[0], 224, 224,3))
    for i in range(len(datasets)):
        datasets_resized[i] =  cv2.resize(datasets_norm[i], (224,224))
    return datasets_resized

In [137]:
def get_inputs_and_labels(datasets, labels):
    #get 2 inputs and corresponding label of the inputs. If the 2 are same class , label is 1. otherwise is 0.
    pair_datasets = np.zeros_like(datasets)
    new_labels = np.zeros_like(labels)
    for i in range(len(datasets)-1):
        pair_datasets[i] = datasets[i+1]
        if labels[i] == labels[i+1]:
            new_labels[i] = [1]
    pair_datasets[len(datasets)-1] = datasets[0]
    if labels[len(datasets)-1] == labels[0]:
        new_labels[len(datasets)-1] = [1]
    return datasets, pair_datasets, new_labels
        
    

In [138]:
def get_input_and_labels(train = True):
    if train:
        ori_data, ori_label = get_5000_training_data()
    else:
        ori_data, ori_label = get_1000_test_data()
        
    return preprocess_data(ori_data), to_categorical(ori_label)